
**Install requirements**

In [0]:
!pip3 install 'torch==1.3.1'
!pip3 install 'torchvision==0.4.2'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'
!pip install --upgrade pillow

Requirement already up-to-date: pillow in /usr/local/lib/python3.6/dist-packages (6.2.1)


**Import libraries**

In [0]:
import os
import os.path
import sys
import logging
import math

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import alexnet
from torchvision.models import vgg11
from torchvision.datasets import VisionDataset
from torchvision import get_image_backend

from PIL import Image
from tqdm import tqdm

**Code From GitHub**

In [0]:
def pil_loader(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

def has_file_allowed_extension(filename, extensions):
    #Returns True if the filename ends with one of given extensions
    return filename.lower().endswith(extensions)

def default_loader(path):
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:
        return pil_loader(path)

def make_dataset(dir, class_to_idx, extensions=None, is_valid_file=None):
    images = []
    dir = os.path.expanduser(dir)
    if not ((extensions is None) ^ (is_valid_file is None)):
        raise ValueError("Both extensions and is_valid_file cannot be None or not None at the same time")
    if extensions is not None:
        def is_valid_file(x):
            return has_file_allowed_extension(x, extensions)
    for target in sorted(class_to_idx.keys()):
        d = os.path.join(dir, target)
        if not os.path.isdir(d):
            continue
        for root, _, fnames in sorted(os.walk(d)):
            for fname in sorted(fnames):
                path = os.path.join(root, fname)
                if is_valid_file(path):
                    item = (path, class_to_idx[target])
                    images.append(item)
    return images


class Caltech(VisionDataset):
    def __init__(self, root, split='train', transform=None, target_transform=None):
        super(Caltech, self).__init__(root, transform=transform, target_transform=target_transform)

        self.split = split # This defines the split you are going to use
                           # (split files are called 'train.txt' and 'test.txt')

        classes, class_to_idx = self._find_classes(self.root)
        IMG_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')
        samples = make_dataset(self.root, class_to_idx, extensions=IMG_EXTENSIONS)

        if len(samples) == 0:
            raise (RuntimeError("Found 0 files in subfolders of: " + self.root + "\n"
                                "Supported extensions are: " + ",".join(IMG_EXTENSIONS)))

        self.loader = default_loader
        self.extensions = IMG_EXTENSIONS
        self.classes = classes
        self.class_to_idx = class_to_idx
        self.samples = samples
        self.targets = [s[1] for s in samples]

    def _find_classes(self, dir):
        if sys.version_info >= (3, 5):
            classes = [d.name for d in os.scandir(dir) if d.is_dir()]
        else:
            classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
        classes.sort()
        classes.remove("BACKGROUND_Google")
        class_to_idx = {classes[i]: i for i in range(len(classes))}
        return classes, class_to_idx
    
    def __getitem__(self, index):
        path, label = self.samples[index]
        image = self.loader(path)
        if self.transform is not None:
            image = self.transform(image)
        if self.target_transform is not None:
            label = self.target_transform(label)
        return image, label

    def __len__(self):
        length = len(self.samples)
        return length

**Set Arguments**

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

NUM_CLASSES = 101 # 101 + 1: There is am extra Background class that should be removed 

BATCH_SIZE = 256     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results
#BATCH_SIZE = 16     # used for VGG

MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

**Define Data Preprocessing**

In [0]:
def make_transform(mean, std):
  # Compose is used to chain more than one transformations
  # Define transforms for training phase  
  train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                        transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                    # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                    # Remember this when applying different transformations, 
                                                                    # otherwise you get an error
                                        transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                        transforms.Normalize((mean[0], mean[1], mean[2]), (std[0], std[1], std[2])) # Normalizes tensor with mean and standard deviation
  ])

  # Define transforms for the validation phase
  valid_transform = transforms.Compose([transforms.Resize(256),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize((mean[0], mean[1], mean[2]), (std[0], std[1], std[2])) 
  ])

  # Define transforms for the final training before evaluation (train + validation)
  tv_transform = transforms.Compose([transforms.Resize(256),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize((mean[0], mean[1], mean[2]), (std[0], std[1], std[2]))                                   
  ])
  
  # Define transforms for the evaluation phase
  eval_transform = transforms.Compose([transforms.Resize(256),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize((mean[0], mean[1], mean[2]), (std[0], std[1], std[2]))                                    
  ])
  return train_transform, valid_transform, tv_transform, eval_transform

mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]
train_transform, valid_transform, tv_transform, eval_transform = make_transform(mean, std)

**Prepare Dataset**

In [0]:
def make_datasets(train_transform, valid_transform, tv_transform, eval_transform):
  # Clone github repository with data
  if not os.path.isdir('./Homework2-Caltech101'):
    !git clone https://github.com/MachineLearning2020/Homework2-Caltech101.git

  # Prepare Pytorch train/test Datasets
  # All the images of the dataset are loaded for both test and train set
  # with the associated transformation (that are the same for both sets)
  #Each image is characterized by a tensor of 3x224x224 elements
  DATA_DIR = 'Homework2-Caltech101/101_ObjectCategories'
  tv_dataset = Caltech(root = DATA_DIR, transform=tv_transform)
  train_dataset = Caltech(root = DATA_DIR, transform=train_transform)
  validation_dataset = Caltech(root = DATA_DIR, transform=valid_transform)
  test_dataset = Caltech(root = DATA_DIR, transform=eval_transform)
    
  f = open ('Homework2-Caltech101/test.txt', 'r')
  test = []
  for line in f:
    test.append(line[:-1])

  # The indexes of the images for train and test sets are choosen
  tv_indexes = []
  train_indexes = []
  test_indexes = []
  validation_indexes = []
  for i in range(len(tv_dataset)):
    if tv_dataset.samples[i][0][len(DATA_DIR)+1:] in test:
      test_indexes.append(i)
    else:
      tv_indexes.append(i)
  tv_dataset = Subset(tv_dataset, tv_indexes)
  test_dataset = Subset(test_dataset, test_indexes)

  # Half of the indexes in train set are used to build teh validation set
  # They are also removed from the train indexes
  change = False
  for i in tv_indexes:
    if change:
      validation_indexes.append(i)
      change = False
    else:
      train_indexes.append(i)
      change = True

  train_dataset = Subset(train_dataset, train_indexes)
  validation_dataset = Subset(validation_dataset, validation_indexes)

  # Check dataset sizes
  print('Train Dataset: {}'.format(len(train_dataset)))
  print('Validation Dataset: {}'.format(len(validation_dataset)))
  print('TV Dataset: {}'.format(len(tv_dataset)))
  print('Test Dataset: {}'.format(len(test_dataset)))

  return train_dataset, validation_dataset, tv_dataset, test_dataset

train_dataset, validation_dataset, tv_dataset, test_dataset = make_datasets(train_transform, valid_transform, tv_transform, eval_transform)

Train Dataset: 2892
Validation Dataset: 2892
TV Dataset: 5784
Test Dataset: 2893


**Prepare Dataloaders**

In [0]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
tv_dataloader = DataLoader(tv_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

**Prepare and Train the Network**

In [0]:
def get_params(net, freeze):
  # Choose parameters to optimize
  # To access a different set of parameters, you have to access submodules of AlexNet
  # (nn.Module objects, like AlexNet, implement the Composite Pattern)
  # e.g.: parameters of the fully connected layers: net.classifier.parameters()
  # e.g.: parameters of the convolutional layers: look at alexnet's source code ;) 
  if freeze == None:
    parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet
  elif freeze == "conv":
    parameters_to_optimize = net.features.parameters()
    for p in parameters_to_optimize:
      p.requires_grad = False
    parameters_to_optimize = net.classifier.parameters()
  elif freeze == "full":
    parameters_to_optimize = net.classifier.parameters()
    for p in parameters_to_optimize:
      p.requires_grad = False
    parameters_to_optimize = net.features.parameters()
  return parameters_to_optimize

def training_phase(train_dataloader, validation_dataloader, validation_dataset, comb, epochs=200, pretrained=False, freeze=None):
  best_accuracy = 0

  for i in range(len(comb["lr"])):

    net = alexnet(pretrained=pretrained) # Loading AlexNet model
    #net = vgg11(pretrained=pretrained)

    # AlexNet has 1000 output neurons, corresponding to the 1000 ImageNet's classes
    # We need 101 outputs for Caltech-101
    net.classifier[6] = nn.Linear(4096, NUM_CLASSES) # nn.Linear in pytorch is a fully connected layer
                                                    # The convolutional layer is nn.Conv2d

    # We just changed the last layer of AlexNet with a new fully connected layer with 101 outputs
    # It is mandatory to study torchvision.models.alexnet source code
    
    print("--------------------------------------") 
    print ("START WITH LEARNING RATE = " + str(comb["lr"][i]) + " AND STEP SIZE = " + str(comb["ss"][i]))
    print("--------------------------------------") 
    b = False;

    # Define loss function
    criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

    parameters_to_optimize = get_params(net, freeze)

    # Define optimizer
    # An optimizer updates the weights based on loss
    # We use SGD with momentum
    optimizer = optim.SGD(parameters_to_optimize, lr=comb["lr"][i], momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

    # Define scheduler
    # A scheduler dynamically changes learning rate
    # The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=comb["ss"][i], gamma=GAMMA)

    # By default, everything is loaded to cpu
    net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

    cudnn.benchmark # Calling this optimizes runtime

    current_step = 0
    # Start iterating over the epochs
    early_stop = [-2, -1, 0]
    for epoch in range(epochs):
      print('\nStarting epoch {}/{}, LR = {}'.format(epoch+1, epochs, scheduler.get_lr()))
      # Iterate over the dataset
      for images, labels in train_dataloader:
        # Bring data over the device of choice
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        net.train() # Sets module in training mode

        # PyTorch, by default, accumulates gradients after each backward pass
        # We need to manually set the gradients to zero before starting a new iteration
        optimizer.zero_grad() # Zero-ing the gradients

        # Forward pass to the network
        outputs = net(images)

        # Compute loss based on output and ground truth
        loss = criterion(outputs, labels)
        if (math.isnan(loss)):
          print ("Loss is NaN!")
          b = True;
          break;

        # Log loss
        if current_step % LOG_FREQUENCY == 0:
          print('Step {}, Loss {}'.format(current_step, loss.item()))

        # Compute gradients for each layer and update weights
        loss.backward()  # backward pass: computes gradients
        optimizer.step() # update weights based on accumulated gradients

        current_step += 1
      
      if b:
        break;
      
      # Step the scheduler
      scheduler.step()

      net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
      net.train(False) # Set Network to evaluation mode

      running_corrects = 0
      for images, labels in tqdm(validation_dataloader):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        # Forward Pass
        outputs = net(images)

        # Get predictions
        _, preds = torch.max(outputs.data, 1)

        # Update Corrects
        running_corrects += torch.sum(preds == labels.data).data.item()

      # Calculate Accuracy
      accuracy = running_corrects / float(len(validation_dataset))
      print('\nValidation Accuracy: {}'.format(accuracy))
      if accuracy > best_accuracy:
        best_accuracy = accuracy
        result = [epoch+1, {"lr": comb["lr"][i], "ss":comb["ss"][i]}]
        print("New best net found!!!")
      if epoch % 10 == 0:
        early_stop[0] = early_stop[1]
        early_stop[1] = early_stop[2]
        early_stop[2] = accuracy
        if early_stop[2] <= early_stop[1] and early_stop[1] <= early_stop[0]:
          print("EARLY STOP!")
          break;
  return result

comb = {"lr":[0.1,0.01,0.01,0.001], "ss":[20, 30, 45, 60]} 
#result = training_phase(train_dataloader, validation_dataloader, validation_dataset, comb)
print (result)

[59, {'lr': 0.01, 'ss': 30}]


**Final Training over Train + Validation**

In [0]:
def final_training(tv_dataloader, comb, epochs=200, pretrained=False, freeze=None):
  net = alexnet(pretrained=pretrained)
  #net = vgg11(pretrained=pretrained)
  net.classifier[6] = nn.Linear(4096, NUM_CLASSES) 
  b = False;
  criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

  parameters_to_optimize = get_params(net, freeze)

  optimizer = optim.SGD(parameters_to_optimize, lr=comb["lr"], momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
  scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=comb["ss"], gamma=GAMMA)
  net = net.to(DEVICE) 
  cudnn.benchmark

  current_step = 0
  for epoch in range(epochs):
    print('\nStarting epoch {}/{}, LR = {}'.format(epoch+1, epochs, scheduler.get_lr()))
    for images, labels in tv_dataloader:
      images = images.to(DEVICE)
      labels = labels.to(DEVICE)
      net.train() 
      optimizer.zero_grad() # Zero-ing the gradients
      outputs = net(images)  
      loss = criterion(outputs, labels)
      if (math.isnan(loss)):
        print ("Loss is NaN!")
        b = True;
        break;
      if current_step % LOG_FREQUENCY == 0:
        print('Step {}, Loss {}'.format(current_step, loss.item()))
      loss.backward() 
      optimizer.step()
      current_step += 1
    if b:
      break;
    scheduler.step()
      
  return net

#best_net = final_training(tv_dataloader, result[1], epochs=result[0])

**Test**

In [0]:
def evaluating_phase(best_net, test_dataloader, test_dataset):

  best_net = best_net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
  best_net.train(False) # Set Network to evaluation mode

  running_corrects = 0
  for images, labels in tqdm(test_dataloader):
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    # Forward Pass
    outputs = best_net(images)

    # Get predictions
    _, preds = torch.max(outputs.data, 1)

    # Update Corrects#
    running_corrects += torch.sum(preds == labels.data).data.item()

  # Calculate Accuracy
  accuracy = running_corrects / float(len(test_dataset))

  print('Test Accuracy: {}'.format(accuracy))

#evaluating_phase(best_net, test_dataloader, test_dataset)

**Transfer Learning**

In [0]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
train_transform, valid_transform, tv_dataset, eval_transform = make_transform(mean, std)
train_dataset, validation_dataset, tv_dataset, test_dataset = make_datasets(train_transform, valid_transform, tv_transform, eval_transform)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
tv_dataloader = DataLoader(tv_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

comb = {"lr":[0.1,0.01,0.01,0.001], "ss":[20, 30, 45, 60]} 
# Pretrained AlexNet
result = training_phase(train_dataloader, validation_dataloader, validation_dataset, comb, pretrained=True)
best_net = final_training(tv_dataloader, result[1], epochs=result[0], pretrained=True)
evaluating_phase(best_net, test_dataloader, test_dataset)
# Pretrained AlexNet freezing convolutional layers
print("\n-----------------------")
print("OTTIMIZZO I CLASSIFIER!")
print("-----------------------\n")
best_net = final_training(tv_dataloader, result[1], epochs=result[0], pretrained=True, freeze="conv")
evaluating_phase(best_net, test_dataloader, test_dataset)
# Pretrained AlexNet freezing fully connected layers
print("\n-----------------------")
print("OTTIMIZZO I FEATURES!")
print("-----------------------\n")
best_net = final_training(tv_dataloader, result[1], epochs=result[0], pretrained=True, freeze="full")
evaluating_phase(best_net, test_dataloader, test_dataset)

Train Dataset: 2892
Validation Dataset: 2892
TV Dataset: 5784
Test Dataset: 2893


Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/checkpoints/alexnet-owt-4df8aa71.pth

  0%|          | 0.00/233M [00:00<?, ?B/s]
  1%|          | 2.44M/233M [00:00<00:09, 25.4MB/s]
  2%|▏         | 5.19M/233M [00:00<00:09, 26.3MB/s]
  3%|▎         | 7.62M/233M [00:00<00:09, 25.9MB/s]
  4%|▍         | 10.2M/233M [00:00<00:08, 26.1MB/s]
  6%|▌         | 12.9M/233M [00:00<00:08, 26.5MB/s]
  6%|▋         | 15.0M/233M [00:00<00:09, 25.0MB/s]
  7%|▋         | 17.1M/233M [00:00<00:10, 21.8MB/s]
  8%|▊         | 19.6M/233M [00:00<00:09, 22.9MB/s]
 10%|▉         | 22.4M/233M [00:00<00:09, 24.3MB/s]
 11%|█         | 25.1M/233M [00:01<00:08, 25.5MB/s]
 12%|█▏        | 27.6M/233M [00:01<00:08, 25.1MB/s]
 13%|█▎        | 30.1M/233M [00:01<00:08, 25.6MB/s]
 14%|█▍        | 32.8M/233M [00:01<00:07, 26.3MB/s]
 15%|█▌        | 35.4M/233M [00:01<00:07, 26.4MB/s]
 16%|█▋        | 38.1M/233M [00:01<00:07, 26.8MB/s]
 17%|█▋        | 40.6M/233M [00:01<00:07,

--------------------------------------
START WITH LEARNING RATE = 0.1 AND STEP SIZE = 20
--------------------------------------

Starting epoch 1/100, LR = [0.1]
Step 0, Loss 4.903846740722656
Loss is NaN!
--------------------------------------
START WITH LEARNING RATE = 0.01 AND STEP SIZE = 30
--------------------------------------

Starting epoch 1/100, LR = [0.01]
Step 0, Loss 5.075348377227783



 33%|███▎      | 4/12 [00:07<00:20,  2.57s/it]Exception ignored in: <bound method Image.__del__ of <PIL.Image.Image image mode=RGB size=300x200 at 0x7F4F004C0B00>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/PIL/Image.py", line 630, in __del__
    self.__exit__()
  File "/usr/local/lib/python3.6/dist-packages/PIL/Image.py", line 592, in __exit__


KeyboardInterrupt: ignored

    if hasattr(self, "fp") and getattr(self, "_exclusive_fp", False):
KeyboardInterrupt




**Data Augmentation**

In [0]:
valid_transform = transforms.Compose([transforms.Resize(256),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))                                    
  ])
eval_transform = transforms.Compose([transforms.Resize(256),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))                                    
  ])
comb = {"lr":[0.1,0.01,0.01,0.001], "ss":[20, 30, 45, 60]}

#FIRST COMBINATION
print("::::::::::::::::::::")
print("FIRST COMBINATION")
print("::::::::::::::::::::")
train_transform = transforms.Compose([transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
                                        transforms.ColorJitter(),
                                        transforms.RandomVerticalFlip(),     
                                        transforms.CenterCrop(224),  
                                        transforms.ToTensor(), 
                                        transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
  ])

tv_transform = transforms.Compose([transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
                                        transforms.ColorJitter(),
                                        transforms.RandomVerticalFlip(),     
                                        transforms.CenterCrop(224),  
                                        transforms.ToTensor(), 
                                        transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
  ])

train_dataset, validation_dataset, tv_dataset, test_dataset = make_datasets(train_transform, valid_transform, tv_transform, eval_transform)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
tv_dataloader = DataLoader(tv_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

result = training_phase(train_dataloader, validation_dataloader, validation_dataset, comb, pretrained=True)
best_net = final_training(tv_dataloader, result[1], epochs=result[0], pretrained=True)
evaluating_phase(best_net, test_dataloader, test_dataset)

# SECOND COMBINATION
print(":::::::::::::::::::::")
print("SECOND COMBINATION")
print(":::::::::::::::::::::")
train_transform = transforms.Compose([transforms.RandomRotation(degrees=30),
                                        transforms.Grayscale(3),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.Resize(256),     
                                        transforms.CenterCrop(224),  
                                        transforms.ToTensor(), 
                                        transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
  ])

tv_transform = transforms.Compose([transforms.RandomRotation(degrees=30),
                                        transforms.Grayscale(3),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.Resize(256),     
                                        transforms.CenterCrop(224),  
                                        transforms.ToTensor(), 
                                        transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
  ])

train_dataset, validation_dataset, tv_dataset, test_dataset = make_datasets(train_transform, valid_transform, tv_transform, eval_transform)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
tv_dataloader = DataLoader(tv_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

result = training_phase(train_dataloader, validation_dataloader, validation_dataset, comb, pretrained=True)
best_net = final_training(tv_dataloader, result[1], epochs=result[0], pretrained=True)
evaluating_phase(best_net, test_dataloader, test_dataset)

# THIRD COMBINATION
print("::::::::::::::::::::")
print("THIRD COMBINATION")
print("::::::::::::::::::::")
train_transform = transforms.Compose([transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
                                        transforms.RandomRotation(degrees=30),
                                        transforms.RandomChoice([transforms.ColorJitter(),transforms.Grayscale(3)]),
                                        transforms.RandomChoice([transforms.RandomHorizontalFlip(),transforms.RandomVerticalFlip()]),     
                                        transforms.CenterCrop(224),  
                                        transforms.ToTensor(), 
                                        transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
  ])

tv_transform = transforms.Compose([transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
                                        transforms.RandomRotation(degrees=30),
                                        transforms.RandomChoice([transforms.ColorJitter(),transforms.Grayscale(3)]),
                                        transforms.RandomChoice([transforms.RandomHorizontalFlip(),transforms.RandomVerticalFlip()]),     
                                        transforms.CenterCrop(224),  
                                        transforms.ToTensor(), 
                                        transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
  ])

train_dataset, validation_dataset, tv_dataset, test_dataset = make_datasets(train_transform, valid_transform, tv_transform, eval_transform)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
tv_dataloader = DataLoader(tv_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

result = training_phase(train_dataloader, validation_dataloader, validation_dataset, comb, pretrained=True)
best_net = final_training(tv_dataloader, result[1], epochs=result[0], pretrained=True)
evaluating_phase(best_net, test_dataloader, test_dataset)

::::::::::::::::::::
THIRD COMBINATION
::::::::::::::::::::
Train Dataset: 2892
Validation Dataset: 2892
TV Dataset: 5784
Test Dataset: 2893
--------------------------------------
START WITH LEARNING RATE = 0.1 AND STEP SIZE = 20
--------------------------------------

Starting epoch 1/100, LR = [0.1]
Step 0, Loss 4.9788384437561035
Loss is NaN!
--------------------------------------
START WITH LEARNING RATE = 0.01 AND STEP SIZE = 30
--------------------------------------

Starting epoch 1/100, LR = [0.01]
Step 0, Loss 4.927743434906006




  0%|          | 0/12 [00:00<?, ?it/s]

  8%|▊         | 1/12 [00:05<01:01,  5.56s/it]

 17%|█▋        | 2/12 [00:06<00:40,  4.06s/it]

 25%|██▌       | 3/12 [00:06<00:26,  2.91s/it]

 33%|███▎      | 4/12 [00:06<00:16,  2.10s/it]

 42%|████▏     | 5/12 [00:10<00:19,  2.72s/it]

 50%|█████     | 6/12 [00:11<00:12,  2.10s/it]

 58%|█████▊    | 7/12 [00:11<00:07,  1.55s/it]

 67%|██████▋   | 8/12 [00:11<00:04,  1.14s/it]

 75%|███████▌  | 9/12 [00:14<00:04,  1.63s/it]

 83%|████████▎ | 10/12 [00:15<00:02,  1.26s/it]

100%|██████████| 12/12 [00:15<00:00,  1.11it/s]




Validation Accuracy: 0.5079529737206085
New best net found!!!

Starting epoch 2/100, LR = [0.01]




  0%|          | 0/12 [00:00<?, ?it/s]

  8%|▊         | 1/12 [00:05<01:01,  5.59s/it]

 17%|█▋        | 2/12 [00:05<00:39,  3.98s/it]

 25%|██▌       | 3/12 [00:06<00:25,  2.86s/it]

 33%|███▎      | 4/12 [00:06<00:16,  2.07s/it]

 42%|████▏     | 5/12 [00:10<00:19,  2.81s/it]

 50%|█████     | 6/12 [00:11<00:12,  2.03s/it]

 58%|█████▊    | 7/12 [00:11<00:07,  1.48s/it]

 67%|██████▋   | 8/12 [00:11<00:04,  1.09s/it]

 75%|███████▌  | 9/12 [00:14<00:04,  1.60s/it]

 83%|████████▎ | 10/12 [00:14<00:02,  1.15s/it]

 92%|█████████▏| 11/12 [00:14<00:00,  1.13it/s]

100%|██████████| 12/12 [00:14<00:00,  1.23s/it]


Validation Accuracy: 0.700207468879668
New best net found!!!

Starting epoch 3/100, LR = [0.01]


...


Validation Accuracy: 0.8084370677731674

Starting epoch 81/100, LR = [0.0001]




  0%|          | 0/12 [00:00<?, ?it/s]

  8%|▊         | 1/12 [00:05<01:04,  5.85s/it]

 17%|█▋        | 2/12 [00:06<00:41,  4.15s/it]

 25%|██▌       | 3/12 [00:06<00:26,  2.97s/it]

 33%|███▎      | 4/12 [00:06<00:17,  2.14s/it]

 42%|████▏     | 5/12 [00:11<00:20,  2.94s/it]

 50%|█████     | 6/12 [00:11<00:12,  2.10s/it]

 58%|█████▊    | 7/12 [00:11<00:07,  1.51s/it]

 67%|██████▋   | 8/12 [00:11<00:04,  1.12s/it]

 75%|███████▌  | 9/12 [00:15<00:05,  1.80s/it]

 92%|█████████▏| 11/12 [00:15<00:01,  1.28s/it]

100%|██████████| 12/12 [00:15<00:00,  1.29s/it]


Validation Accuracy: 0.8077455048409405
EARLY STOP!

Starting epoch 1/54, LR = [0.001]
Step 0, Loss 4.982235908508301

Starting epoch 2/54, LR = [0.001]

Starting epoch 3/54, LR = [0.001]

Starting epoch 4/54, LR = [0.001]

Starting epoch 5/54, LR = [0.001]
Step 100, Loss 1.086017370223999

Starting epoch 6/54, LR = [0.001]

Starting epoch 7/54, LR = [0.001]

Starting epoch 8/54, LR = [0.001]

Starting epoch 9/54, LR = [0.001]

Starting epoch 10/54, LR = [0.001]
Step 200, Loss 0.6291376948356628

Starting epoch 11/54, LR = [0.001]

Starting epoch 12/54, LR = [0.001]

Starting epoch 13/54, LR = [0.001]

Starting epoch 14/54, LR = [0.001]
Step 300, Loss 0.5481369495391846

Starting epoch 15/54, LR = [0.001]

Starting epoch 16/54, LR = [0.001]

Starting epoch 17/54, LR = [0.001]

Starting epoch 18/54, LR = [0.001]

Starting epoch 19/54, LR = [0.001]
Step 400, Loss 0.34616607427597046

Starting epoch 20/54, LR = [0.001]

Starting epoch 21/54, LR = [0.001]

Starting epoch 22/54, LR = [0.00



  0%|          | 0/12 [00:00<?, ?it/s]

  8%|▊         | 1/12 [00:07<01:20,  7.33s/it]

 17%|█▋        | 2/12 [00:07<00:52,  5.21s/it]

 25%|██▌       | 3/12 [00:07<00:33,  3.72s/it]

 33%|███▎      | 4/12 [00:08<00:21,  2.68s/it]

 42%|████▏     | 5/12 [00:12<00:21,  3.06s/it]

 50%|█████     | 6/12 [00:12<00:13,  2.21s/it]

 58%|█████▊    | 7/12 [00:12<00:08,  1.62s/it]

 67%|██████▋   | 8/12 [00:12<00:04,  1.18s/it]

 75%|███████▌  | 9/12 [00:14<00:03,  1.32s/it]

 92%|█████████▏| 11/12 [00:14<00:00,  1.05it/s]

100%|██████████| 12/12 [00:14<00:00,  1.22s/it]

Test Accuracy: 0.8492913930176288
